In [1]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'processed'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [3]:
ct = pd.read_csv(src / 'ct-from-test-data' / 'ct.csv', index_col='date_sample')
ct_by_awh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-awh.csv')
ct_by_adh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-adh.csv')
ct_by_sex = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-sex.csv')
ct_by_ag = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-ag.csv')
addiv = pd.read_csv(ref / 'addiv.csv')[['id_addiv', 'name_addiv_2']]
pop = pd.read_csv(ref / 'pop_1.csv')[['id_addiv', 'pop']]

In [4]:
ct

,no_test,no_positive,no_positive_group_sample,no_test_corrected,pct_positive,no_test_rollsum,no_positive_rollsum,pct_positive_per7d,ct
date_sample,,,,,,,,,
2020-07-04,2.0,0.0,0.0,2.0,0.000000,NaN,NaN,0.000000,1
2020-07-05,46.0,4.0,0.0,46.0,0.086957,NaN,NaN,0.000000,1
2020-07-06,1.0,0.0,0.0,1.0,0.000000,NaN,NaN,0.000000,1
2020-07-07,281.0,21.0,0.0,281.0,0.074733,NaN,NaN,0.000000,1
2020-07-08,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,0.000000,1
...,...,...,...,...,...,...,...,...,...
2021-09-07,23734.0,6091.0,959.0,22775.0,0.267442,141674.0,44245.0,0.312301,4
2021-09-08,19277.0,5433.0,566.0,18711.0,0.290364,142644.0,42652.0,0.299010,4
2021-09-09,18366.0,5296.0,542.0,17824.0,0.297127,142632.0,42120.0,0.295305,4


In [5]:
tphcm = {
    'addr_dist_home':['THANH PHO HO CHI MINH'],
    'no_test_corrected': [ct[['no_test_corrected', 'no_positive']].sum()[0]],
    'no_positive': [ct[['no_test_corrected', 'no_positive']].sum()[1]]
}

df_tphcm = pd.DataFrame(tphcm)

df = (
    ct_by_adh[['date_sample', 'addr_dist_home', 'no_test_corrected', 'no_positive']]
#     .query('date_sample == "2021-08-05"')
    .groupby('addr_dist_home')[['no_test_corrected', 'no_positive']]
    .sum()
#     .to_frame(name='no_test_cumsum')
    .reset_index()
    .append(df_tphcm)
    .sort_values('no_test_corrected')
    .merge(
        addiv,
        how = 'left',
        left_on = 'addr_dist_home',
        right_on = 'name_addiv_2'
    )
    .merge(
        pop,
        how = 'left',
        left_on = 'id_addiv',
        right_on = 'id_addiv'
    )
    .drop(columns={'id_addiv', 'name_addiv_2'})
)

df['pct_positive'] = df['no_positive'] / df['no_test_corrected']

df['predicted_inc'] = df['pop'] * df['pct_positive']

df.to_csv(src / 'predicted-inc-from-test-data' / 'predicted-inc.csv', index=False)
df

,addr_dist_home,no_test_corrected,no_positive,pop,pct_positive,predicted_inc
0,HUYEN CAN GIO,17898.0,2011.0,71526,0.112359,8.036584e+03
1,QUAN PHU NHUAN,19668.0,5734.0,163961,0.291540,4.780112e+04
2,QUAN 06,20881.0,11234.0,233561,0.538001,1.256561e+05
3,QUAN 02,22439.0,4546.0,180275,0.202594,3.652258e+04
4,QUAN 11,22791.0,7647.0,209867,0.335527,7.041608e+04
5,HUYEN NHA BE,27454.0,14418.0,206837,0.525169,1.086245e+05
6,QUAN 05,32629.0,12724.0,159073,0.389960,6.203208e+04
7,QUAN 01,34753.0,14110.0,142625,0.406008,5.790691e+04
8,QUAN 10,39155.0,18919.0,234819,0.483182,1.134604e+05
9,QUAN 07,41483.0,15758.0,360155,0.379866,1.368108e+05


In [6]:
# df = (
#     ct_by_adh[['addr_dist_home', 'date_sample', 'no_test', 'no_positive']]
#     .groupby(['addr_dist_home', 'date_sample']).sum()
#     .groupby(level=0).cumsum().reset_index()
#     .merge(
#         addiv,
#         how = 'left',
#         left_on = 'addr_dist_home',
#         right_on = 'name_addiv_2'
#     )
#     .merge(
#         pop,
#         how = 'left',
#         left_on = 'id_addiv',
#         right_on = 'id_addiv'
#     )
#     .drop(columns={'id_addiv', 'name_addiv_2'})
# )

# df['pct_positive'] = df['no_positive'] / df['no_test']

# df['predicted_inc'] = df['pop'] * df['pct_positive']
# # foo[foo.addr_dist_home == 'HUYEN BINH CHANH'].tail(10)
# df